In [3]:
# Import libraries                                                                                                                                                                                                                                                                                                                                                                                                                        import pandas as pd 
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

In [4]:
# url = "https://www.zomato.com/mumbai/great-food-no-bull"
# path = r'C:\Users\Deepthi Poojary\Web Scrapping\chromedriver'
# drive = webdriver.Chrome(path)
# drive.implicitly_wait(30)
# drive.get(url)
# source = drive.page_source

In [5]:
# py_button = drive.find_elements_by_css_selector("div[class = 'col-s-8 col-l-1by3']") #.get_attribute("href")
# print(len(py_button))

In [6]:
# Open tab in full screen (maximized)
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")

In [7]:
# Instantiate Selenium Webdriver
driver = webdriver.Chrome(options = options , executable_path = r'C:\Users\Deepthi Poojary\Web Scrapping\chromedriver')

# Get the main page (Top Restaurants in Mumbai on Zomato)
driver.get("https://www.zomato.com/mumbai/great-food-no-bull")

# Get page soure
source = driver.page_source

In [8]:
# Get the main page/window handle after it has opened
main_window = driver.window_handles[0]

In [9]:
# Get title of  main window
main_window_title = driver.title
print(main_window_title)

Best Food in Mumbai | Zomato


In [10]:
# Calculating number of restaurant on the main page using BeautifulSoup and main page source
soup = BeautifulSoup(source,"html.parser") # col-s-8 col-l-1by3
type(soup)
button = soup.find_all("div", attrs = {"class":"col-s-8 col-l-1by3"})
bs_len = len(button)
bs_len

186

This is the number of restaurant currently on the main page.

In [11]:
# Calculating number of restaurants on the main page using Selenium's find_elements_by_css_selector 
links = driver.find_elements_by_css_selector("div[class = 'col-s-8 col-l-1by3']") #driver.find_element_by_css_selector("div[data_res_position = '1']")  #WebDriverWait(driver,20).until(EC.element_to_be_clickable(("div[data-res-position = '1']")))
sel_len = len(links)
sel_len

186

This is just to confirm the number of restaurants on main page and hence bs_len should be equal to sel_len.

In [12]:
type(links)

list

In [13]:
# # selecting a particular restaurant from main page
# py_button = drive.find_elements_by_css_selector("div[data-res-position = '1']") #.get_attribute("href")
# print(len(py_button))
# #print(py_button)
# #print(py_button.text)
# #print(py_button.get_attribute("a"))

In [14]:
#  Data we'll be extracting
# Made a dictionary for ease of structuring dataframe
table = {"Name":[],"Area":[],"Type":[],"Rating":[],"Ph_Number":[],"Cuisine":[],"Avg_price":[],"Beer":[],"Time":[],"Adress":[],"Facility":[],"Food_Rating":[],"Service_Rating":[],"Ambience_Rating":[] }

In [15]:
def subs(val):
    if val is None:
        val = np.NAN
    return val

In [ ]:
for link in links:        # for loop loops through links of all restaurants in links variable

    
    ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()        # Opens each restaurants link in new tab by  "ctrl" + clicking (link) 
    
    # get the window handle after a new window/tab has opened
    new_window = driver.window_handles[1]
    
    # Switch focus to new window so that required operations can be done in that tab
    driver.switch_to.window(new_window)
    
    # get new window title
    new_window_title = driver.title
    
    if main_window_title != new_window_title:  # Check if main_window and new_window are different
                    
        
        # Providing page source of new window
        Source = driver.page_source
        
        

        # BeautifulSoup 
        soup1 = BeautifulSoup(Source, "html.parser")
        outlet = driver.find_elements_by_css_selector("div[class = 'card  search-snippet-card     search-card  ']") 
##### For Resaturant having more than one outlet
        if outlet:
            optional = driver.find_elements_by_css_selector("div[class= 'search_left_featured clearfix']")
            for l in optional:
                ##ActionChains(driver).key_down(Keys.CONTROL).click(l).key_up(Keys.CONTROL).perform()
                l.click()       # Clicking on the link and opening on the same page
                ##new_window1 = driver.window_handles[2]
                ##driver.switch_to.window(new_window1)
                Source2 = driver.page_source
                soup2 = BeautifulSoup(Source2,'html.parser')


                ### Gate 1
                header = soup2.find("div", attrs =  {"class" : "col-l-12"})
                # Restaurant Name
                Name = soup2.find("div", attrs = {"class" : "col-l-12"}).h1.text.strip('\n').strip(" ")
                Name = subs(Name)
                table["Name"].append(Name)
                # Main adress
                Area = header.div.a.text.strip(" ").strip("\n").strip(" ")
                Area = subs(Area)
                table["Area"].append(Area)
                # Type of Restaurant
                Type = header.find("span", attrs = {"class" : "res-info-estabs grey-text fontsize3"}).text.strip("\n")
                Type = subs(Type)
                table["Type"].append(Type)


                ### Gate 2
                header2 = soup2.find("div"  , attrs = {"class" : "col-l-4 tac left"})
                #Rating of restaurent
                Rating = header2.find("div" , attrs = {"tabindex" : "0"}).text.strip().replace(" ","").replace("\n","")
                Rating = subs(Rating)
                table["Rating"].append(Rating)


                ### Gate 3
                tail1 = soup2.find("div" , class_ = "col-l-1by3 pl0 pr20")
                #Phone Number 
                Ph_Number = tail1.find("span" , attrs = {"class" : "fontsize2 bold zgreen"}).text
                Ph_Number = subs(Ph_Number)
                table["Ph_Number"].append(Ph_Number)
                #Cusines
                Cuisine = tail1.find("div" , class_ = "res-info-cuisines clearfix").text
                Cuisine = subs(Cuisine)
                table["Cuisine"].append(Cuisine)
                #Average  price or price for two person
                Avg_price = tail1.find("div" , class_ = "res-info-detail").find("span" ,attrs = {"tabindex" : "0"}).text.split("for")[0]
                Avg_price = subs(Avg_price)
                table["Avg_price"].append(Avg_price)
                #Beer price/pint
                Beer = tail1.find("div" , class_ = "res-info-detail").find("div",class_ = "mt5").text.strip("\n").split(" ")[0]
                Beer = subs(Beer)
                table["Beer"].append(Beer)


                ### Gate 4
                tail2= soup2.find("div",attrs={"class" : "col-l-1by3 pl20 pr20"})
                # Today's timing
                Time = tail2.find("div",attrs= {"class":"clearfix"}).find("div",class_ = "medium").text.strip("Today")
                Time = subs(Time)
                table["Time"].append(Time)
                # Full Adress
                Address = tail2.find("div" , class_ = "resinfo-icon").span.text
                Address = subs(Address)
                table["Adress"].append(Address)

                # Using "What people love here" to get food rating, service rating and ambience rating.
                ### Gate 5
                tail_end = soup2.find_all("div" , attrs = {"class" , "col-l-1by3 pl20 pr20"})[1]
                #Additional_Info
                Facility = tail_end.find("div",attrs={"class" : "res-info-highlights"}).text.strip("\n").replace("\n"," ").replace("More Info","").strip()
                Facility = subs(Facility)
                table["Facility"].append(Facility)
                # Food rating
                exp = soup2.find_all("div" , class_ = "rv_highlights__score_bar mt5 mb5")
                length = len(exp)

                lis1 = exp[0].find_all("div" , class_ = "block level-8")
                n1 = 0
                for element in lis1:
                    n1 += 1
                food_rating = n1
                table["Food_Rating"].append(food_rating)

                if length >= 2:
                    # Service rating
                    lis2 = exp[1].find_all("div" , class_ = "block level-7")
                    n2 = 0
                    for element in lis2:
                        n2 += 1
                    service_rating = n2
                    table["Service_Rating"].append(service_rating)



                    if length == 3:       # Some restaurant don't have data for ambience rating

                        # Ambience Rating
                        lis3 = exp[2].find_all("div" , class_ = "block level-6")
                        n3 = 0
                        for element in lis3:
                            n3+=1
                        ambience_rating = n3
                        table["Ambience_Rating"].append(ambience_rating)
                    else:              # if no ambience rating put NA 
                        hue =np.NAN
                        table["Ambience_Rating"].append(hue)
                     #Ask Selenium to click the back button
                else:
                    na = np.NAN
                    table["Service_Rating"].append(na)
                    table["Ambience_Rating"].append(na)
                    
                    #drive.execute_script("window.history.go(-1)") 
                ##driver.close
                driver.execute_script("window.history.go(-1)")
                ##driver.switch_to.window(new_window)
                
        else:
            ### Gate 1
                header = soup1.find("div", attrs =  {"class" : "col-l-12"})
                # Restaurant Name
                Name = soup1.find("div", attrs = {"class" : "col-l-12"}).h1.text.strip('\n').strip(" ")
                Name = subs(Name)
                table["Name"].append(Name)
                # Main adress
                Area = header.div.a.text.strip(" ").strip("\n").strip(" ")
                Area = subs(Area)
                table["Area"].append(Area)
                # Type of Restaurant
                Type = header.find("span", attrs = {"class" : "res-info-estabs grey-text fontsize3"}).text.strip("\n")
                Type = subs(Type)
                table["Type"].append(Type)


                ### Gate 2
                header2 = soup1.find("div"  , attrs = {"class" : "col-l-4 tac left"})
                #Rating of restaurent
                Rating = header2.find("div" , attrs = {"tabindex" : "0"}).text.strip().replace(" ","").replace("\n","")
                Rating = subs(Rating)
                table["Rating"].append(Rating)


                ### Gate 3
                tail1 = soup1.find("div" , class_ = "col-l-1by3 pl0 pr20")
                #Phone Number 
                Ph_Number = tail1.find("span" , attrs = {"class" : "fontsize2 bold zgreen"}).text
                Ph_Number = subs(Ph_Number)
                table["Ph_Number"].append(Ph_Number)
                #Cusines
                Cuisine = tail1.find("div" , class_ = "res-info-cuisines clearfix").text
                Cuisine = subs(Cuisine)
                table["Cuisine"].append(Cuisine)
                #Average  price or price for two person
                Avg_price = tail1.find("div" , class_ = "res-info-detail").find("span" ,attrs = {"tabindex" : "0"}).text.split("for")[0]
                Avg_price = subs(Avg_price)
                table["Avg_price"].append(Avg_price)
                #Beer price/pint
                Beer = tail1.find("div" , class_ = "res-info-detail").find("div",class_ = "mt5").text.strip("\n").split(" ")[0]
                Beer = subs(Beer)
                table["Beer"].append(Beer)


                ### Gate 4
                tail2= soup1.find("div",attrs={"class" : "col-l-1by3 pl20 pr20"})
                # Today's timing
                Time = tail2.find("div",attrs= {"class":"clearfix"}).find("div",class_ = "medium").text.strip("Today")
                Time = subs(Time)
                table["Time"].append(Time)
                # Full Adress
                Address = tail2.find("div" , class_ = "resinfo-icon").span.text
                Address = subs(Address)
                table["Adress"].append(Address)

                # Using "What people love here" to get food rating, service rating and ambience rating.
                ### Gate 5
                tail_end = soup1.find_all("div" , attrs = {"class" , "col-l-1by3 pl20 pr20"})[1]
                #Additional_Info
                Facility = tail_end.find("div",attrs={"class" : "res-info-highlights"}).text.strip("\n").replace("\n"," ").replace("More Info","").strip()
                Facility = subs(Facility)
                table["Facility"].append(Facility)
                # Food rating
                exp = soup1.find_all("div" , class_ = "rv_highlights__score_bar mt5 mb5")
                length = len(exp)

                lis1 = exp[0].find_all("div" , class_ = "block level-8")
                n1 = 0
                for element in lis1:
                    n1 += 1
                food_rating = n1
                table["Food_Rating"].append(food_rating)

                if length >= 2:
                    # Service rating
                    lis2 = exp[1].find_all("div" , class_ = "block level-7")
                    n2 = 0
                    for element in lis2:
                        n2 += 1
                    service_rating = n2
                    table["Service_Rating"].append(service_rating)



                    if length == 3:       # Some restaurant don't have data for ambience rating

                        # Ambience Rating
                        lis3 = exp[2].find_all("div" , class_ = "block level-6")
                        n3 = 0
                        for element in lis3:
                            n3+=1
                        ambience_rating = n3
                        table["Ambience_Rating"].append(ambience_rating)
                    else:              # if no ambience rating put NA 
                        hue =np.NAN
                        table["Ambience_Rating"].append(hue)
                else:
                    na = np.NAN
                    table["Service_Rating"].append(na)
                    table["Ambience_Rating"].append(na)
                     #Ask Selenium to click the back button
                    #drive.execute_script("window.history.go(-1)") 
                
        
        

            # Close new tab after getting all data
        driver.close()


            # Switch handle back to main tab to open next restaurant's link
        driver.switch_to.window(main_window)

In [17]:
len(table)

14

In [18]:
table       # take a look at dictionary (including 14 sample of restaurant data)

{'Name': ['Kettle & Keg - Dining & Bar',
  'MH04 The Drunk Yard',
  'The Finch',
  'Kaitlyns Beer Garden',
  'Vanilla Beans',
  'Curry And Combos Twist',
  'Culinary Tales',
  'Trèsind',
  'Glocal Kitchen',
  'Smiley Pops',
  'Kebabs And Curries Company',
  'The Cheaters',
  'Lemon Leaf',
  'On Toes',
  'Curry Patta'],
 'Area': ['7 Bungalows, Andheri West',
  'Manpada, Thane West',
  'Powai',
  'Hill Road, Bandra West',
  'Bandra Talao, Bandra West',
  'Versova, Andheri West',
  'Veera Desai Area',
  'Bandra Kurla Complex',
  'Kandivali East',
  'Andheri Lokhandwala, Andheri West',
  'Kurla',
  'Marol',
  'Bandra Talao, Bandra West',
  'Veera Desai Area',
  'Bandra Talao, Bandra West'],
 'Type': ['Casual Dining, Bar ',
  'Bar ',
  'Microbrewery, Casual Dining ',
  'Pub, Casual Dining ',
  'Bakery ',
  'Quick Bites ',
  'Casual Dining ',
  'Fine Dining ',
  'Casual Dining ',
  'Dessert Parlor, Quick Bites ',
  'Casual Dining ',
  'Casual Dining, Bar ',
  '                               

In [20]:
# First 14 Restaurant extracted fromthe site
dataset = pd.DataFrame(table)
dataset

,Name,Area,Type,Rating,Ph_Number,Cuisine,Avg_price,Beer,Time,Adress,Facility,Food_Rating,Service_Rating,Ambience_Rating
0,Kettle & Keg - Dining & Bar,"7 Bungalows, Andheri West","Casual Dining, Bar",4.7/5,022 26319696,"Cafe, Continental, Seafood, Italian, Fast Food","₹1,800",₹185,9:30am – 1am,"Shop 3/4, Opposite Marin Vue Building, J P Roa...",Home Delivery Full Bar Available Live Music...,5,4.0,0.0
1,MH04 The Drunk Yard,"Manpada, Thane West",Bar,4.5/5,022 61739793,"Finger Food, North Indian, Italian, Mexican, C...","₹1,500",₹209,12noon – 1am,"C-1 B-1, Kothari Compound, 27 Acre, Beside Roy...",Full Bar Available Live Sports ScreeningLive ...,5,0.0,0.0
2,The Finch,Powai,"Microbrewery, Casual Dining",4.6/5,022 50645610,"North Indian, Continental, Italian, Rolls, Chi...","₹2,500",₹450,12noon – 1:30am,"Shah Industrial Estate, Opposite Huntsman Buil...",Home Delivery Wheelchair Accessible Full B...,5,0.0,0.0
3,Kaitlyns Beer Garden,"Hill Road, Bandra West","Pub, Casual Dining",4.5/5,022 61344960,"Finger Food, Mexican, Italian, Continental, Pi...","₹2,200",₹185,"12midnight – 1:30am, 12noon – 12midnight","202, Khan House, Hill Road, Bandra West, Mumbai",Full Bar Available Live Sports ScreeningValet...,5,4.0,4.0
4,Vanilla Beans,"Bandra Talao, Bandra West",Bakery,4.5/5,022 68493624,"Bakery, Desserts",₹750,Cash\n,10am – 9pm,"16, Araadhna Apartments, Gurunanak Road, Bandr...",Home Delivery Seating Not Available Home Ba...,0,0.0,NaN
5,Curry And Combos Twist,"Versova, Andheri West",Quick Bites,4.5/5,+91 7506647826,"North Indian, Chinese, Fast Food, Pizza, Panin...",₹550,Cash\n,9:30am – 12midnight,"Shop 22, New Sangam Society, Juhu Versova Link...",Home Delivery Seating Not Available,5,NaN,NaN
6,Culinary Tales,Veera Desai Area,Casual Dining,4.5/5,022 30151690,"Chinese, European, Continental, Salad, Italian...","₹1,700",Cash\n,10am – 1am,"Shop 7-8,Bhavesha Apartments, Near Azad Nagar ...",Home Delivery Free ParkingIndoor SeatingTable...,5,4.0,4.0
7,Trèsind,Bandra Kurla Complex,Fine Dining,4.5/5,022 33126978,Modern Indian,"₹4,000",Cash\n,"12noon – 3:30pm, 7pm – 12midnight","Ground Floor, Inspire BKC, G Block, Opposite A...",Full Bar Available Valet Parking AvailableInd...,5,0.0,0.0
8,Glocal Kitchen,Kandivali East,Casual Dining,4.5/5,022 50645805,"Continental, North Indian, Thai, Chinese","₹1,000",Cash\n,"11am – 4pm, 7pm – 11pm","Shop 24, Vasant Utsav, Rakhee Building, Thakur...",Indoor Seating,5,0.0,NaN
9,Smiley Pops,"Andheri Lokhandwala, Andheri West","Dessert Parlor, Quick Bites",4.5/5,022 50646680,"Desserts, Ice Cream, Beverages",₹700,Cash\n,9am – 1am,"Royal Classic Building, New Link Road, Near Ci...",Home Delivery Vegetarian Only Outdoor Seati...,5,0.0,0.0
